In [ ]:
!pip install mne
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.filters.git
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.utils.git
!pip install bctpy
!pip install pingouin

In [ ]:
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y análisis
# ==============================================================================
import statsmodels.api as sm
import pingouin as pg
from scipy import stats
from scipy.stats import pearsonr
from scipy.signal import butter, lfilter, welch, hanning, filtfilt, hilbert
# Configuración matplotlib
# ==============================================================================
# plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
#-----------------------------------------------------------------------------------------------------------------
import pickle
import numpy as np
from scipy.io import loadmat,savemat
from mne.preprocessing import ICA
# from google.colab import drive
import tensorflow_probability as tfp
from sklearn.model_selection import KFold
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import euclidean_distances
import mne
#-----------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy.spatial.distance import squareform 
import matplotlib.colors as colors
import matplotlib.cm as cmx
from gcpds.utils.visualizations import plot_eeg, plot_topoplot
# from mne.viz.topomap import _check_outlines, _draw_outlines
from mne.viz.utils import plt_show,tight_layout
from mne.io.pick import _picks_to_idx
from mne.viz import plot_topomap
#-----------------------------------------------------------------------------------------------------------------
from sklearn.metrics import pairwise_distances  
from scipy.spatial.distance import squareform 
from scipy.stats import kurtosis
import pywt
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import periodogram, welch
from scipy.stats import spearmanr
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mne.time_frequency import tfr_array_morlet, csd_array_morlet, csd_array_fourier
from joblib import Parallel, delayed
from scipy.stats import kendalltau
import scipy.io as sio
import numpy as np
from joblib import load
import bct
#-----------------------------------------------------------------------------------------------------------------
# -*- coding: utf-8 -*-
import tensorflow as tf
import matplotlib.pyplot as plt
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow_probability as tfp
from sklearn.decomposition import PCA
from tensorflow.keras import regularizers
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.model_selection import train_test_split
import types
import tempfile
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
# Sujetos de la base de datos
Subjects = list(np.arange(1,22))
# canales de la base de datos
channels = ['Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3','Cz','C4','T4',
            'T5','P3','Pz','P4','T6','O1','O2']
n_channels = len(channels)
# Frecuencia de muestreo de la base de datos.
sampling_freq  = 1000
# Número de trials en resting
N_tr = 20
# ubicación de la base de datos.
load_path = '../input/bcmi-music/Data 1/'
# Método utilizado para la función de ICA de MNE.
method = 'fastica'
# Número de núcleos para ell multiproceso
# pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
# ciclo de los sujetos.
y = list()
data_ref = list()
data_ref_ica = list()
for s in Subjects:
  # carga de la información del sujetos.
  S_ERP = loadmat(load_path + f"P_"+str(s)+"/P"+str(s)+"_BCMI_frontHN_2017.mat")
  print('sujeto: ', s)
  data_m = S_ERP['music'][0][0][0]
  data_m = np.transpose(data_m, (2,1,0)) # transponer en trials x canales x tiempo
  data_target_m = data_m
  data_r = S_ERP['base']
  data_r = np.transpose(data_r, (1,0)) # transponer en canales x tiempo
  data_r = data_r[:n_channels,10*sampling_freq:-10*sampling_freq]
  data_target_r = np.zeros((N_tr,data_r.shape[0],9500))
  step = 9.5*sampling_freq
  windows = list(np.arange(0,data_r.shape[1],step))
  for tr in range(N_tr):
    data_target_r[tr,:,:] = data_r[:,int(windows[tr]):int(windows[tr]+step)]
  data_target = np.concatenate([data_target_m,data_target_r],axis=0)
  # data_target = data_target_m

  # structure mne datos en (trials,canales,tiempo)
  ch_types = ['eeg']*n_channels
  info = mne.create_info(channels, ch_types=ch_types, sfreq=sampling_freq)
  info.set_montage('standard_1005')
  raw = mne.EpochsArray(data_target, info)

  # señal sin quitar los artefactos.
  # plt.figure(figsize=(20,8))
  # plot_eeg(data_target_m[0,:,:],channels,sampling_freq)
  # plt.title('Señal trial 1 de música-sin filtrar')

  # filtro la señal de 0.5 - 100 Hz.
  raw.filter(0.5,100)

  # plt.figure(figsize=(20,8))
  # plot_eeg(raw.get_data()[0,:,:],channels,sampling_freq)
  # plt.title('Señal trial 1 de música-filtrada de 0.5 a 100 Hz')

  # plt.figure(figsize=(20,8))
  # plot_eeg(data_target_r[0,:,:],channels,sampling_freq)
  # plt.title('Señal trial 1 de resting-sin filtrar')

  # plt.figure(figsize=(20,8))
  # plot_eeg(raw.get_data()[13,:,:],channels,sampling_freq)
  # plt.title('Señal trial 1 de música-filtrada de 0.5 a 100 Hz')

  # Repairing artifacts with ICA
  # ica.plot_sources(raw, show_scrollbars=False)
  # ica = ICA(n_components=n_channels, method=method, max_iter='auto', random_state=0)
  # ica.fit(raw)
  # componentes = ica.get_sources(raw).get_data()
  
  # # señal sin quitar los artefactos.
  # plt.figure(figsize=(20,8))
  # plot_eeg(componentes[0,:,:],channels,sampling_freq)
  
  # Parametros para el análisis de la entropia.
  # tau     = 10              # 20   30 
  # window  = 1*sampling_freq # 3,   3   segundo de la ventana.
  # step    = window*0.5      # 50%, 80% de traslape.
  # windows = list(np.arange(0,componentes.shape[2]-window,step))

  # start_time = time.time()
  # entropy_tr = list()
  # for tr in range(data_target.shape[0]):
  #   data = list()
  #   for time_ in range(len(windows)):
  #     signal = np.squeeze(raw.get_data()[tr,:,int(windows[time_]):int(windows[time_]+window)])
  #     data.append(signal)
  #   # estimación de la entropía de la señal.
  #   entropy_tr.append(np.asarray(pool.map(fun_entropy, data)))
  # entropy_raw = np.asarray(entropy_tr)
  # print('Tiempo de la entropia de componentes {tim}'.format(tim=(time.time()-start_time)))
  
  # ica.plot_sources(mne.io.RawArray(raw.get_data()[0], info))
  
  # start_time = time.time()
  # entropy_tr = list()
  # for tr in range(data_target.shape[0]):
  #   data = list()
  #   for time_ in range(len(windows)):
  #     signal = np.squeeze(componentes[tr,:,int(windows[time_]):int(windows[time_]+window)])
  #     data.append(signal)
  #   # estimación de la entropía de las componentes.
  #   entropy_tr.append(np.asarray(pool.map(fun_entropy, data)))
  # entropy_com = np.asarray(entropy_tr)
  # print('Tiempo de la entropia de componentes {tim}'.format(tim=(time.time()-start_time)))
  
  # raw2 = raw.copy()
  # comp_ = []
  # for tr in range(entropy_com.shape[0]):
  #   datos_ = np.max(entropy_com[tr,:,:],axis=0)-np.min(entropy_com[0,:,:],axis=0)
  #   datos_f = np.asarray([round(a,1) for a in datos_])
  #   comp_.append(np.where((datos_f>0)==True)[0])
  # pos = list(np.unique(np.concatenate(comp_)))
  # pos.extend([0,1])
  # ica.exclude = [0,1]          # indices chosen based on various plots above sujeto 2.
  # ica.apply(raw2)
  
  # figura señal quitando artefactos
  # plt.figure(figsize=(20,8))
  # plot_eeg(raw2.get_data()[0,:,:],channels,sampling_freq)

  # start_time = time.time()
  # entropy_tr = list()
  # for tr in range(data_target.shape[0]):
  #   data = list()
  #   for time_ in range(len(windows)):
  #     signal = np.squeeze(raw2.get_data()[tr,:,int(windows[time_]):int(windows[time_]+window)])
  #     data.append(signal)
  #   # estimación de la entropía de la señal quitando artefactos.
  #   entropy_tr.append(np.asarray(pool.map(fun_entropy, data)))
  # entropy_ica = np.asarray(entropy_tr)
  # print('Tiempo de la entropia de la señal {tim}'.format(tim=(time.time()-start_time)))

  # CAR common average reference
  # En la señal.
  rereferenced_raw, ref_data = mne.set_eeg_reference(raw, ref_channels='average',projection=True,verbose=0)
  # data_ref.append(rereferenced_raw.get_data())
  data_ref.append(rereferenced_raw.get_data())
  # En la señal reconstruida.
  # rereferenced_raw_ica, ref_data = mne.set_eeg_reference(raw2, ref_channels='average',projection=True,verbose=0)
  # data_ref_ica.append(rereferenced_raw.get_data())
  # data_ref_ica.append(rereferenced_raw_ica.get_data())
  # savemat(load_path+f'Resultado2_music_rest_Sujeto'+str(s)+'.mat',{'X_ica':raw2.get_data(),'X_raw':raw.get_data(),'ref_raw':data_ref,
  #           'ref_ica':data_ref_ica,'H_raw':entropy_raw,'H_com':entropy_com,'H_ica':entropy_ica})

# PSD por ventanas de tiempo de 1 seg

In [ ]:
# Calculate PSD
nblock = 1000
win = hanning(nblock, True)
window = 1*sampling_freq
step = 0.1*sampling_freq # traslape 90%
windows = list(np.arange(0,data_ref[0].shape[2]-window,step))
data_power_ = list()

for sub in range(len(Subjects)):
  Power = list()
  for tr in range(data_ref[sub].shape[0]):
    Power_ven = list()
    for ven in range(len(windows)):
      freqs, Pxxf = welch(np.squeeze(data_ref[sub][tr,:,int(windows[ven]):int(windows[ven]+window)]), sampling_freq, window=win,  nfft=nblock, return_onesided=True, scaling='spectrum')
      Power_ven.append(Pxxf[:,8:13])
    Power.append(np.asarray(Power_ven))
  data_power_.append(np.asarray(Power))
Data_pow = np.asarray(data_power_)

## ACC anteriores

In [ ]:
resultado_j = loadmat('../input/main-music/Main_music.mat')
Acc_ = np.mean(resultado_j['facc_'],axis=1)
Acc_.shape

# ACC Nuevos

In [ ]:
resultado_j = loadmat('../input/mainmusic2/Main_music2.mat')
Acc_ = np.mean(resultado_j['facc_'],axis=1)
Acc_.shape

# Organizar características y acierto 

In [ ]:
met = 1
# Acc = np.zeros()
if met == 1:
  Acc = Acc_
  datos = np.reshape(Data_pow,(Data_pow.shape[0],Data_pow.shape[1]*
                               Data_pow.shape[2]*Data_pow.shape[3]*
                               Data_pow.shape[4]))
elif met == 2:
  # datos = np.zeros((21,912000))
  Acc = np.zeros((21,1))
  index = list()
  for sub in range(len(Acc_)):
    index.append(resultado_j['facc_'][sub,:]>=Acc_[sub])
  index_ = np.sum(np.asarray(index),axis=0).reshape(-1,1).T>10
  Acc = np.mean(resultado_j['facc_'][:,index_.reshape(-1)],axis=1)
  datos_ = Data_pow[:,:,index_.reshape(-1),:,:]
  datos = np.reshape(datos_,(datos_.shape[0],datos_.shape[1]*datos_.shape[2]*
                             datos_.shape[3]*datos_.shape[4]))
else:
  datos = np.zeros((21,912000))
  Acc = np.zeros((21,1))
  index = list()
  tam = list()
  for sub in range(len(Acc_)):
    index_ = resultado_j['facc_'][sub,:]>=Acc_[sub]
    Acc[sub] = np.mean(resultado_j['facc_'][sub,index_])
    datos_ = Data_pow[sub,:,index_.reshape(-1),:,:]
    temp = np.reshape(datos_,(datos_.shape[0]*datos_.shape[1]*datos_.shape[2]*
                             datos_.shape[3]))
    tam.append(temp.shape)
    datos[sub,0:temp.shape[0]] = temp

In [ ]:
XT = datos
YT = Acc.reshape(-1,1) # agrego el acc de los sujetos
YT.shape

# normalizar ACC y filas de las características

In [ ]:
#----------------------------------------------------------------------------------------------------------------
ind_YT = np.argsort(Acc).reshape(-1)
# for each sample (row) we apply min-max transformation
XT_norm_ = np.zeros((XT.shape))
for k in range(XT.shape[0]):
  XT_norm_[k,:] = (XT[k,:]-XT[k,:].min())/(XT[k,:].max()-XT[k,:].min())
XT_norm = XT_norm_[ind_YT,:]
#----------------------------------------------------------------------------------------------------------------
# for each sample (row) we apply min-max transformation
# YTa = np.zeros((YT_.shape))
# for k in range(YT_.shape[0]):
#   YTa[k,:] = (YT_[k,:]-YT_[k,:].min())/(YT_[k,:].max()-YT_[k,:].min())
# YT = YTa[ind_YT,:]
YT = YT[ind_YT]
YT.shape

# regresion SVR

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import LeaveOneOut, GridSearchCV, train_test_split, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from sklearn import svm, preprocessing
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin, RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
#this one to prevent warning about gamma settings
# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
# data loading scaling, splitting
# validation subject
best_ = list()
results_ = list()

scorer = make_scorer(mean_squared_error)#, greater_is_better=False)
# 30 trials donde se realiza una pertición de 3 grupos iguales, donde los dos primeros
# se utilizaron para entrenamiento y el último de validación 
# clases de la base de datos.
y = YT
# PSD
X = XT_norm
# PAC in the time.
# dat = np.squeeze()
# X = np.reshape(dat,(dat.shape[0],dat.shape[1]*dat.shape[2]))
# Grid_s = list()
# for i in range(50):
# print(f'fold '+str(i))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=0)
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.3,random_state=0)
# shuffle_split = StratifiedKFold(n_splits=2, random_state=0, shuffle=False)
# cv = 
steps = [('Reg', SVR(kernel='rbf'))]
pipe_svr = Pipeline(steps)
params_grid={
          'Reg__C': [0.01, 0.1, 1, 100, 1000],
          'Reg__gamma':[0.01, 0.1, 1, 100, 1000]
          }

grid = GridSearchCV(estimator=pipe_svr, 
                  param_grid=params_grid, 
                  scoring=scorer,
                  cv =shuffle_split,
                  # LeaveOneOut(),
                  error_score='raise',
                  verbose=40,n_jobs=-1)

grid.fit(X,y.ravel())
results_.append(grid.best_estimator_.predict(X))#(grid.cv_results_)
best_.append(grid.best_estimator_)

In [ ]:
MAE = list()
R2  = list()
for sub in range(len(best_)):
    y_true,y_pred = YT,results_[sub]
    mae = mean_absolute_error(y_true,y_pred)
    r2  = r2_score(y_true,y_pred)
    MAE.append(mae)
    R2.append(r2)
    print('Sujeto {sub}: Promedio MAE es {mae}, y R2 es {r2}'.format(sub=sub+1,mae=mae,r2=r2))
print('Sujetos: Promedio MAE es {mae}, y R2 es {r2}'.format(mae=np.mean(MAE),r2=np.mean(R2)))

# PAC

In [ ]:
#filtering functions
def butter_bandpass(lowcut, highcut, fs, order=4):
  #lowcut is the lower bound of the frequency that we want to isolate
  #hicut is the upper bound of the frequency that we want to isolate
  #fs is the sampling rate of our data
  nyq = 0.5 * fs #nyquist frequency - see http://www.dspguide.com/ if you want more info
  low = float(lowcut) / nyq
  high = float(highcut) / nyq
  b, a = butter(order, [low, high], btype='band')
  return b, a

def butter_bandpass_filter(mydata, lowcut, highcut, fs, order=4):
  b, a = butter_bandpass(lowcut, highcut, fs, order=order)
  y = filtfilt(b, a, mydata)
  return y

In [ ]:
from numpy import angle, sin, cos, sqrt
def moving_multitrial_pac(data1,data2,fr1,fr2,sampling_freq,type):
  #calculating phase of theta
  phase_data = butter_bandpass_filter(data1, fr1[0], fr1[1],(sampling_freq))
  phase_data = angle(hilbert(phase_data))

  #calculating amplitude envelope of high gamma
  amp_data = butter_bandpass_filter(data2, fr2[0], fr2[1], (sampling_freq))
  amp_data = abs(hilbert(amp_data))

  # Filtering the amplitude of analytic signal of the high-frequency range within the
  # frequency range of the low-frequency band
  lowfromhigh = butter_bandpass_filter(amp_data, fr1[0], fr1[1],(sampling_freq))
  
  # lowfromhigh.shape
  low_Env_high_filtered_signals = lowfromhigh-np.tile(np.mean(lowfromhigh),(Ntrials,1))
  
  # get the phase
  Amp_phase_signals=angle(hilbert(low_Env_high_filtered_signals))
  Ntime = 1*sampling_freq #data1.shape[1]
  Nstep = 0.1*sampling_freq
  windows = list(np.arange(0,data_ref[0].shape[2]-window,step))
  Nsegments = len(windows)

  if type == 'trials':
    temporal_ = np.zeros((Amp_phase_signals.shape[0],Nsegments))
  else:
    temporal_ = np.zeros((Nsegments))
  for ii in range(Nsegments): # repear over different segments
    start = int(windows[ii])
    stop  = int(windows[ii]+Ntime)
    if type == 'trials':
      plv = np.abs(np.sum(np.exp(1j*(phase_data[:,start:stop]-Amp_phase_signals[:,start:stop])),axis=1)/((Nstep)))
      temporal_[:,ii]=plv
    else:
      plv = np.abs(np.sum(np.sum(np.exp(1j*(phase_data[:,start:stop]-Amp_phase_signals[:,start:stop]))))/(Ntrials*(Nstep)))
      temporal_[ii]=plv
  return temporal_

# parámetros de PAC

In [ ]:
# Parameters
# Bands of rytms: delta to gamma.
phase_providing_band = [[8,12]] 
amplitude_providing_band = [[8,12]]
Ntrials   = data_ref[0].shape[0]
# Nsegments = 10 # Number of segments in window.
Nchannels = data_ref[0].shape[1]
Nfreqs    = len(phase_providing_band)

In [ ]:
#@title Texto de título predeterminado
# temporal_plv = np.zeros((len(Subjects),int(Nchannels*(Nchannels-1)/2),Nfreqs,Nfreqs,Nsegments))
Nsegments=85
temporal_plv = np.zeros((len(Subjects),Ntrials,int(Nchannels*(Nchannels-1)/2),Nfreqs,Nfreqs,Nsegments))
# (Nodes,Nodes,Time)

for sub in range(len(Subjects)):
  cont_ch = 0 # contador de las subredes
  for ch1 in range(Nchannels):
    for ch2 in range(ch1+1,Nchannels-1):
      if ch1 != ch2:
        data1= np.squeeze(data_ref[sub][:,ch1,:])
        data2= np.squeeze(data_ref[sub][:,ch2,:])
        fr_1 = 0
        for fr1 in phase_providing_band:
          fr_2 = 0
          for fr2 in amplitude_providing_band:
            temporal_plv[sub,:,cont_ch,fr_1,fr_2,:] = moving_multitrial_pac(data1,data2,fr1,fr2,sampling_freq,type='trials')
            fr_2 += 1
          fr_1 += 1
      cont_ch += 1
  # savemat(load_path+f'PLV_pac'+str(sub)+'_trials.mat',{'temporal_plv':temporal_plv})

In [ ]:
np.squeeze(temporal_plv).shape

In [ ]:
met = 1
# Acc = np.zeros()
dat = np.squeeze(temporal_plv)
if met == 1:
  Acc = Acc_
  datos = np.reshape(dat,(dat.shape[0],dat.shape[1]*
                               dat.shape[2]*dat.shape[3]))
elif met == 2:
  # datos = np.zeros((21,912000))
  Acc = np.zeros((21,1))
  index = list()
  for sub in range(len(Acc_)):
    index.append(resultado_j['facc_'][sub,:]>=Acc_[sub])
  index_ = np.sum(np.asarray(index),axis=0).reshape(-1,1).T>10
  Acc = np.mean(resultado_j['facc_'][:,index_.reshape(-1)],axis=1)
  datos_ = dat[:,:,:,index_.reshape(-1)]
  datos = np.reshape(datos_,(datos_.shape[0],datos_.shape[1]*datos_.shape[2]*
                             datos_.shape[3]))
else:
  datos = np.zeros((21,912000))
  Acc = np.zeros((21,1))
  index = list()
  tam = list()
  for sub in range(len(Acc_)):
    index_ = resultado_j['facc_'][sub,:]>=Acc_[sub]
    Acc[sub] = np.mean(resultado_j['facc_'][sub,index_])
    datos_ = dat[sub,:,:,:,index_]
    temp = np.reshape(datos_,(datos_.shape[0]*datos_.shape[1]*datos_.shape[2]*
                             datos_.shape[3]))
    tam.append(temp.shape)
    datos[sub,0:temp.shape[0]] = temp

In [ ]:
XT = datos
YT = Acc.reshape(-1,1) # agrego el acc de los sujetos
YT.shape

In [ ]:
#----------------------------------------------------------------------------------------------------------------
ind_YT = np.argsort(Acc).reshape(-1)
# for each sample (row) we apply min-max transformation
XT_norm_ = np.zeros((XT.shape))
for k in range(XT.shape[0]):
  XT_norm_[k,:] = (XT[k,:]-XT[k,:].min())/(XT[k,:].max()-XT[k,:].min())
XT_norm = XT_norm_[ind_YT,:]
#----------------------------------------------------------------------------------------------------------------
# for each sample (row) we apply min-max transformation
# YTa = np.zeros((YT_.shape))
# for k in range(YT_.shape[0]):
#   YTa[k,:] = (YT_[k,:]-YT_[k,:].min())/(YT_[k,:].max()-YT_[k,:].min())
# YT = YTa[ind_YT,:]
YT = YT[ind_YT]
YT.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import LeaveOneOut, GridSearchCV, train_test_split, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from sklearn import svm, preprocessing
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin, RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
#this one to prevent warning about gamma settings
# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
# data loading scaling, splitting
# validation subject
best_ = list()
results_ = list()

scorer = make_scorer(mean_squared_error)#, greater_is_better=False)
# 30 trials donde se realiza una pertición de 3 grupos iguales, donde los dos primeros
# se utilizaron para entrenamiento y el último de validación 
# clases de la base de datos.
y = YT
# PSD
X = XT_norm
# PAC in the time.
# dat = np.squeeze()
# X = np.reshape(dat,(dat.shape[0],dat.shape[1]*dat.shape[2]))
# Grid_s = list()
# for i in range(50):
# print(f'fold '+str(i))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=0)
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.3,random_state=0)
# shuffle_split = StratifiedKFold(n_splits=2, random_state=0, shuffle=False)
# cv = 
steps = [('Reg', SVR(kernel='rbf'))]
pipe_svr = Pipeline(steps)
params_grid={
          'Reg__C': [0.01, 0.1, 1, 100, 1000],
          'Reg__gamma':[0.01, 0.1, 1, 100, 1000]
          }

grid = GridSearchCV(estimator=pipe_svr, 
                  param_grid=params_grid, 
                  scoring=scorer,
                  cv =shuffle_split,
                  # LeaveOneOut(),
                  error_score='raise',
                  verbose=20,n_jobs=-1)

grid.fit(X,y.ravel())
results_.append(grid.best_estimator_.predict(X))#(grid.cv_results_)
best_.append(grid.best_estimator_)

In [ ]:
MAE = list()
R2  = list()
for sub in range(len(best_)):
    y_true,y_pred = YT,results_[sub]
    mae = mean_absolute_error(y_true,y_pred)
    r2  = r2_score(y_true,y_pred)
    MAE.append(mae)
    R2.append(r2)
    print('Sujeto {sub}: Promedio MAE es {mae}, y R2 es {r2}'.format(sub=sub+1,mae=mae,r2=r2))
print('Sujetos: Promedio MAE es {mae}, y R2 es {r2}'.format(mae=np.mean(MAE),r2=np.mean(R2)))

In [ ]:
Sujeto 1: Promedio MAE es 0.05418981481481479, y R2 es 0.24613730264050303
Sujetos: Promedio MAE es 0.05418981481481479, y R2 es 0.24613730264050303

Sujeto 1: Promedio MAE es 0.05418981481481479, y R2 es 0.24613730264050303